## Get GitHub Repository

In [1]:
!git clone https://github.com/tiny-dnn/tiny-dnn.git

Cloning into 'tiny-dnn'...
remote: Enumerating objects: 10110, done.
remote: Total 10110 (delta 0), reused 0 (delta 0), pack-reused 10110
Receiving objects: 100% (10110/10110), 24.98 MiB | 15.52 MiB/s, done.
Resolving deltas: 100% (7192/7192), done.


In [2]:
%cd tiny-dnn/
%pwd

/content/tiny-dnn


'/content/tiny-dnn'

## Compile and Execute
1. delete unnecessary code  
/content/tiny-dnn/examples/CMakeLists.txt  
2. update image reader padding to 0, and update image and label paths  
/content/tiny-dnn/examples/mnist/train.cpp  
```c++
  tiny_dnn::parse_mnist_labels(data_dir_path + "/apex_train_train_labels.idx1",
                               &train_labels);
  tiny_dnn::parse_mnist_images(data_dir_path + "/apex_train_train_images.idx3",
                               &train_images, -1.0, 1.0, 0, 0);
  tiny_dnn::parse_mnist_labels(data_dir_path + "/apex_test_train_labels.idx1",
                               &test_labels);
  tiny_dnn::parse_mnist_images(data_dir_path + "/apex_test_train_images.idx3",
                               &test_images, -1.0, 1.0, 0, 0);
```
3. edit output to 3  
/content/tiny-dnn/examples/mnist/train.cpp   
```c++
  using conv    = tiny_dnn::convolutional_layer;
  using pool    = tiny_dnn::max_pooling_layer;
  using fc      = tiny_dnn::fully_connected_layer;
  using relu    = tiny_dnn::relu_layer;
  using softmax = tiny_dnn::softmax_layer;

  const size_t n_fmaps  = 16;  // number of feature maps for upper layer
  const size_t n_fmaps2 = 32;  // number of feature maps for lower layer
  const size_t n_fc     = 32;  // number of hidden units in fc layer

  nn << conv(32, 32, 3, 1, n_fmaps, tiny_dnn::padding::same, true, 1, 1, 1, 1, backend_type)                      // C1
     << pool(32, 32, n_fmaps, 2, false, backend_type)  // P2
     << relu()                                  // activation
     << conv(16, 16, 3, n_fmaps, n_fmaps2, tiny_dnn::padding::same, true, 1, 1, 1, 1, backend_type)                      // C3
     << pool(16, 16, n_fmaps2, 2, false, backend_type)  // P4
     << relu()                                  // activation                                           // activation
     << fc(8 * 8 * n_fmaps2, n_fc, true, backend_type)    // FC7
     << relu()                                            // activation
     << fc(n_fc, 3, true, backend_type) << softmax(3);  // FC3
```
4. edit test.cpp (original code is wrong)   
/content/tiny-dnn/examples/mnist/test.cpp  
```c++
  // mnist dataset is "white on black", so negate required
  //std::transform(
  //  resized.begin(), resized.end(), std::back_inserter(data),
  //  [=](uint8_t c) { return (255 - c) * (maxv - minv) / 255.0 + minv; });

  data.resize(resized.width() * resized.height() * resized.depth());
  for (size_t c = 0; c < resized.depth(); ++c) {
    for (size_t y = 0; y < resized.height(); ++y) {
      for (size_t x = 0; x < resized.width(); ++x) {
        data[c * resized.width() * resized.height() + y * resized.width() + x] =
          (maxv - minv) * (resized[y * resized.width() + x + c]) / 255.0 + minv;
      }
    }
  }

  // sort & print top-3
  for (int i = 0; i < 3; i++)
    std::cout << i << "," << res[i] << std::endl;
```
5. copy data to data folder  

In [19]:
### if you want to make again:
%cd ..
%pwd
!cmake . -DBUILD_EXAMPLES=ON
!make

/content/tiny-dnn
-- Could not find NNPACK include. Turning NNPACK_FOUND off
-- Could not find NNPACK lib. Turning NNPACK_FOUND off
-- TBB and OMP disabled: Using Pthread instead.
-- C++14 support has been enabled by default.
-- cotire 1.7.9 loaded.
-- Cannot generate Caffe Importer. Please install Protobuf.
-- 
-- ******************* tiny-dnn Configuration Summary *******************
-- General:
--   Version           :   1.0.0
--   System            :   Linux
--   C++ compiler      :   /usr/bin/c++
--   Release CXX flags :   -O3 -DNDEBUG  -O3 -pthread  -msse3 -mavx -Wall -Wpedantic -Wno-narrowing -Wno-deprecated  -msse3 -mavx -Wall -Wpedantic -Wno-narrowing -Wno-deprecated
--   Debug CXX flags   :   -g  -g3 -pthread -pthread  -msse3 -mavx -Wall -Wpedantic -Wno-narrowing -Wno-deprecated  -msse3 -mavx -Wall -Wpedantic -Wno-narrowing -Wno-deprecated
--   Build type        :   Release
-- 
--   BUILD_EXAMPLES    :   ON
--   BUILD_TESTS       :   OFF
--   BUILD_DOCS        :   OFF
--   BUI

## Then change to examples directory and run executable files.

In [20]:
%cd examples/
%pwd

/content/tiny-dnn/examples


'/content/tiny-dnn/examples'

In [21]:
!./example_mnist_train --data_path ../data --learning_rate 0.01 --epochs 30

Running with the following parameters:
Data path: ../data
Learning rate: 0.01
Minibatch size: 16
Number of epochs: 30
Backend type: AVX

load models...
start training

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
**************************************************Epoch 1/30 finished. 1.72344s elapsed.
10/30

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
**************************************************Epoch 2/30 finished. 1.75213s elapsed.
14/30

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
**************************************************Epoch 3/30 finished. 1.69357s elapsed.
21/30

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
**************************************************Epoch 4/30 finished. 1.70274s elapsed.
22/30

0%   10   20   30   

0:apnea, 1:hyponea, 2:normal  

In [24]:
!./example_mnist_test ../data/a0013.jpg # 0
# !./example_mnist_test ../data/h0080.jpg # 1
# !./example_mnist_test ../data/n0043.jpg # 2

0,0.498833
1,0.280488
2,0.220679


https://github.com/tiny-dnn/tiny-dnn/  
https://github.com/tiny-dnn/tiny-dnn/blob/c0f576f5cb7b35893f62127cb7aec18f77a3bcc5/tiny_dnn/io/mnist_parser.h   
https://tiny-dnn.readthedocs.io/en/latest/layers/Layers.html